In [1]:
import torch

In [2]:
torch.__version__

'2.0.1'

### Bert base model approach

In [8]:
from transformers import pipeline

pipe = pipeline("fill-mask", model="bert-base-uncased")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.weight', 'cls.seq_relationship.weight', 'bert.pooler.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
temp = pipe("Finance is a fundamental pillar of the modern global economy, governing the allocation and management of funds. It encompasses a vast spectrum of activities, from personal financial planning to intricate investment strategies for multinational corporations. At its core, finance revolves around the efficient utilization of monetary resources, striving to maximize value and mitigate risk.In the realm of personal finance, individuals make decisions about saving, investing, budgeting, and debt management to secure their financial well-being. Sound financial planning empowers individuals to achieve their life goals, whether it be buying a home, funding education, or planning for retirement. In the corporate world, finance plays a pivotal role in driving growth and sustainability. It involves capital budgeting, risk assessment, and the allocation of resources to various projects. Financial markets, both traditional and digital, enable companies to raise capital and manage liquidity efficiently. In an era of rapid technological advancements, finance has seen an evolution through digitalization and fintech innovations. Mobile banking, cryptocurrencies, and peer-to-peer lending platforms are reshaping the financial landscape, offering greater accessibility and efficiency. Finance, at its heart, is a balancing act between risk and reward. It influences our daily lives, guides corporate strategies, and shapes the global economy. Understanding the principles of finance empowers individuals and organizations to make informed decisions and navigate the intricate world of money, investments, and financial well-being [MASK]")

# Flask App

In [3]:
import os 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
import speech_recognition as sr 
from googletrans import Translator
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModelForSeq2SeqLM
from flask import Flask, request, jsonify
from flask_cors import CORS
from pydub import AudioSegment  
from transformers import pipeline
from moviepy.editor import VideoFileClip

app = Flask(__name__)
CORS(app, origins="http://localhost:3000", supports_credentials=True)

@app.route('/api/convert-video-to-mp3', methods=['POST'])
def convert_video_to_mp3():
    if 'file' not in request.files:
        return jsonify({'error': 'No file provided'}), 400

    video_file = request.files['file']
    if video_file.filename == '':
        return jsonify({'error': 'No selected file'}), 400

    if video_file: 
        try: 
            video_path = os.path.join('./uploads', video_file.filename)  
            print(video_path)
            video_file.save(video_path)
 
            audio_path = os.path.abspath('./uploads/output.mp3') 
            video_clip = VideoFileClip(video_path)
            audio_clip = video_clip.audio
            audio_clip.write_audiofile(audio_path)
            audio_clip.close()
            video_clip.close()

            return jsonify({'message': 'Video converted to MP3 successfully', 'audio_file': audio_path}), 200
        except Exception as e:
            return jsonify({'error': 'Conversion error', 'details': str(e)}), 500
        

@app.route('/api/convert-mp3-to-text', methods=['POST'])
def convert_mp3_to_text():
    if 'file' not in request.files:
        return jsonify({'error': 'No file part'}), 400

    file = request.files['file']
    if file.filename == '':
        return jsonify({'error': 'No selected file'}), 400

    if file: 
        mp3_filename = os.path.join('uploads', file.filename)
        file.save(mp3_filename)
 
        wav_filename = mp3_filename.replace('.mp3', '.wav')
        AudioSegment.from_mp3(mp3_filename).export(wav_filename, format="wav")
 
        recognizer = sr.Recognizer()
        with sr.AudioFile(wav_filename) as source:
            audio = recognizer.record(source)

        try:
            text = recognizer.recognize_google(audio)
            return jsonify({'text': text})
        except sr.UnknownValueError:
            return jsonify({'error': 'Could not understand audio'}), 400
        except sr.RequestError as e:
            return jsonify({'error': f'Speech Recognition error: {e}'}), 500
        

@app.route('/api/convert-mp4-to-text', methods=['POST'])
def convert_mp4_to_text():
    file = os.path.abspath('./uploads/output.mp3') 

    if file: 
        mp3_filename = file
        wav_filename = mp3_filename.replace('.mp3', '.wav') 
        AudioSegment.from_mp3(mp3_filename).export(wav_filename, format="wav")
 
        recognizer = sr.Recognizer()
        with sr.AudioFile(wav_filename) as source:
            audio = recognizer.record(source)

        try:
            text = recognizer.recognize_google(audio)
            return jsonify({'text': text})
        except sr.UnknownValueError:
            return jsonify({'error': 'Could not understand audio'}), 400
        except sr.RequestError as e:
            return jsonify({'error': f'Speech Recognition error: {e}'}), 500
        

@app.route('/api/translate_toar/<string:text>', methods=['GET'])
def translate_to_ar(text):    
    translator = Translator() 
    arabic_translation = translator.translate(text, src='en', dest='ar').text
    return jsonify({'translated_txt': arabic_translation})


@app.route('/api/translate_totr/<string:text>', methods=['GET'])
def translate_to_tr(text):  
    translator = Translator()    
    turkish_translation = translator.translate(text, src='en', dest='tr').text
    return jsonify({'translated_txt': turkish_translation})

@app.route('/api/findtopic/<string:text>', methods=['GET'])
def topic_finder(text):
    pipe = pipeline("summarization", model="google/pegasus-xsum")
    topic = pipe(text, max_length = 100)
    topic = topic[0]['summary_text']
    return jsonify({'topic': topic})

@app.route('/api/findSummary/<string:text>', methods=['GET'])
def summary_find(text):
    pipe = pipeline("summarization", model="google/pegasus-xsum")
    output = pipe(text, max_length = 200)
    final_summary = output[0]['summary_text']

    translator = Translator() 
    arabic_summary = translator.translate(final_summary, src='en', dest='ar').text
    turkish_summary = translator.translate(final_summary, src='en', dest='tr').text

    return jsonify({'summary_en': final_summary, 'summary_ar': arabic_summary, 'summary_tr': turkish_summary})


@app.route('/api/sentiment/<string:text>', methods=['GET'])
def sentiment(text):
    nltk.download('vader_lexicon')
    analyzer = SentimentIntensityAnalyzer()
    scores = analyzer.polarity_scores(text)
    positive_percent = round(scores['pos'] * 100, 2)
    negative_percent = round(scores['neg'] * 100, 2)

    return jsonify({'positive': positive_percent, 'negative': negative_percent})

if __name__ == '__main__': 
    app.run(debug=False)

c:\Users\haris\miniconda3\envs\forPyTorch\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [19/Oct/2023 23:06:41] "POST /api/convert-mp3-to-text HTTP/1.1" 200 -
Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Your max_length is set to 100, but your input_length is only 21. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=10)
127.0.0.1 - - [19/Oct/2023 23:07:41] "GET /api/findtopic/American%20accent%20in%2010%20seconds%20is%20it%20is%20saying%20is%20he%20nice%20say%20easy%20nice%20easy%20nice%20easy%20nice%20nice HTTP/1.1" 200 -
Some weights of PegasusForConditionalGeneration were not initializ